<a href="https://colab.research.google.com/github/ElhamHosseini73/LearnNLP/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# you will be implementing logistic regression for sentiment analysis on tweets.
# Given a tweet, you will decide if it has a positive sentiment or a negative one

#step:
#Learn how to extract features for logistic regression given some text
#Implement logistic regression from scratch
#Apply logistic regression on a natural language processing task
#Test using your logistic regression
#Perform error analysis

In [ ]:
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples

In [ ]:

all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

data = all_positive_tweets +all_negative_tweets

lable_positive = [1]* len(all_positive_tweets)
lable_negative = [0]* len(all_negative_tweets)

In [22]:
#Train test split: 20% will be in the test set, and 80% in the training set.
len_pos_data_train = int(len(all_positive_tweets)*0.8)
len_neg_data_train = int(len(all_negative_tweets)*0.8)

x_train = all_positive_tweets[0:len_pos_data_train] + all_negative_tweets[0:len_neg_data_train]
x_test =  all_positive_tweets[len_pos_data_train:] + all_negative_tweets[len_neg_data_train:]

lable_train = lable_positive[0:len_pos_data_train]+lable_negative[0:len_pos_data_train]
lable_test =  lable_positive[len_pos_data_train:]+lable_negative[len_pos_data_train:]

In [ ]:
#preprocessing twitter

import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import string

stopwords_english = stopwords.words('english')

tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

#Stemming
from nltk.stem import PorterStemmer

# Instantiate stemming class
stemmer = PorterStemmer()

def preprocessing_twitter(text):
  # remove old style retweet text "RT"
  text = re.sub(r'^RT[\s]+', '', text)

  # remove hyperlinks
  text = re.sub(r'https?://[^\s\n\r]+', '', text)

  # remove hashtags
  # only removing the hash # sign from the word
  text = re.sub(r'#', '', text)

  # tokenize tweets
  text_tokens = tokenizer.tokenize(text)

  text_clean = []

  for word in text_tokens: # Go through every word in your tokens list
    if (word not in stopwords_english and  # remove stopwords
        word not in string.punctuation):  # remove punctuation
        text_clean.append(word)

  text_stem = []

  for word in text_clean:
    stem_word = stemmer.stem(word)  # stemming word
    text_stem.append(stem_word)  # append to the list



In [ ]:

# this function will build a dictionary where we can lookup how many times a word appears in the lists of positive or negative tweets.

def build_freqs(tweets, lables):

  #   Input: tweets: a list of tweets && lables: an m x 1 array with the sentiment label of each tweet (either 0 or 1)
  #   Output: freqs: a dictionary mapping each (word, lable) pair to its frequency

  word_freq = {}

  for index , tweet in enumerate(tweets):
    process_tweet = preprocessing_twitter(tweet)
    for token in process_tweet:
      if (token,lables[index]) in word_freq:
        word_freq[token,lables[index]] = word_freq[token,lables[index]]+1
      else:
        word_freq[token,lables[index]] = 1
  return word_freq

In [ ]:
#build_freqs(tweets, lables)